In [1]:
# import package for displaying NER results
from spacy import load, displacy
import pandas as pd
import sqlalchemy
import mysql.connector as mysql
from nltk.tokenize import word_tokenize
import re
import string
import nltk
stopwords_id = nltk.corpus.stopwords.words('indonesian')
from tqdm import tqdm
# Memanggil Library Request
import requests
# Memanggil Library BeautifulSoup
from bs4 import BeautifulSoup
from sqlalchemy.pool import NullPool

In [2]:
# import model
link_to_model = "model"
nlp = load(link_to_model)

In [3]:
# Connect DB
def con_db():
	database_username = 'ubuntu'
	database_password = 'D!5k0m#24'
	database_ip       = '10.200.26.13'
	database_name     = 'boss'

	database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}:3636/{3}'.
												format(database_username, database_password, 
														database_ip, database_name), poolclass=NullPool)
	
	return database_connection

In [4]:
# Get from DB
df_tabel = pd.read_sql(f'SELECT \
						URL, \
						Kategori, \
						Tanggal, \
						Isi \
					FROM pjb \
					WHERE isi IS NOT NULL  '
				, con = con_db())

df_tabel = df_tabel.drop_duplicates(subset=['URL']).reset_index(drop=True)
print(f'Jumlah data yang akan diproses: {len(df_tabel)}')
df_tabel

Jumlah data yang akan diproses: 19432


,URL,Kategori,Tanggal,Isi
0,https://jabar.antaranews.com/berita/426135/pre...,Perekonomian,2023-01-02,Jakarta (ANTARA) - Presiden Joko Widodo (Jokow...
1,https://jabar.antaranews.com/berita/426144/res...,Perekonomian,2023-01-02,Jakarta (ANTARA) - Presiden RI Joko Widodo (Jo...
2,https://jabar.antaranews.com/berita/426309/kur...,Perekonomian,2023-01-03,Jakarta (ANTARA) - Nilai tukar (kurs) rupiah y...
3,https://jabar.antaranews.com/berita/426360/har...,Perekonomian,2023-01-03,Singapura (ANTARA) - Harga minyak bertahan dal...
4,https://jabar.antaranews.com/berita/426423/ema...,Perekonomian,2023-01-04,Chicago (ANTARA) - Harga emas naik tajam pada ...
...,...,...,...,...
19427,https://jabar.tribunnews.com/news/1146711/bere...,Ketenagakerjaan,jabar.tribunnews.com-news-1146711,TRIBUNJABAR.ID - Media sosial tengah diramaika...
19428,https://jabar.tribunnews.com/news/1147406/dibu...,Ketenagakerjaan,jabar.tribunnews.com-news-1147406,TRIBUNJABAR.ID - Bagi kamu yang minat bekerja ...
19429,https://jabar.tribunnews.com/news/1147635/solu...,Ketenagakerjaan,jabar.tribunnews.com-news-1147635,TRIBUNJABAR.ID - Kementerian Agama telah mengu...
19430,https://www.cnbcindonesia.com/news/20250310113...,Ketenagakerjaan,2025-03-10,"Jakarta, CNBC Indonesia - Dua pimpinan Komisi ..."


In [6]:
# TES
text = df_tabel['Isi'][0]

doc = nlp(text)
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])


Entities [('Joko Widodo (Jokowi)', 'PERSON'), ('Pasar Tanah Abang', 'LOCATION'), ('Jakarta Pusat', 'LOCATION'), ('Senin', 'TIME'), ('Jokowi', 'PERSON'), ('Pasar Tanah Abang', 'LOCATION'), ('pukul 9.27 WIB', 'TIME'), ('Jokowi', 'PERSON'), ('DKI Jakarta', 'LOCATION'), ('Heru Budi Hartono', 'PERSON'), ('Pasar Tanah Abang Blok A', 'LOCATION'), ('Jokowi', 'PERSON'), ('Indonesia', 'LOCATION'), ('PPKM', 'ORGANIZATION'), ('Jumat (30/12)."Karena 2022', 'TIME'), ('PPKM', 'ORGANIZATION'), ('Jokowi', 'PERSON'), ('PPKM', 'ORGANIZATION'), ('Palembang', 'LOCATION'), ('Pasar Terong', 'LOCATION'), ('Makassar', 'LOCATION'), ('Pasar Beringharjo', 'LOCATION'), ('Yogyakarta', 'LOCATION'), ('Pasar Turi', 'LOCATION'), ('Surabaya', 'LOCATION'), ('Pasar Johar', 'LOCATION'), ('Semarang', 'LOCATION'), ('Pasar Gedebage', 'LOCATION'), ('Bandung', 'LOCATION'), ('Jokowi', 'PERSON'), ('Pasar Tanah Abang', 'LOCATION'), ('Jokowi', 'PERSON'), ('RI', 'LOCATION'), ('Joko Widodo', 'PERSON'), ('Indonesia', 'LOCATION'), ('Jo

In [7]:
list_data = []

for index, row in tqdm(df_tabel.iterrows()):
	kat = row['Kategori']
	isi = row['Isi']
	doc = nlp(isi)
	for ent in doc.ents:
		row = [kat, ent.text, ent.label_]
		list_data.append(row)
	


19432it [06:44, 48.02it/s] 


In [27]:
df = pd.DataFrame(list_data, columns=['Kat','Entitas','Label'])
df

,Kat,Entitas,Label
0,Perekonomian,Joko Widodo (Jokowi),PERSON
1,Perekonomian,Pasar Tanah Abang,LOCATION
2,Perekonomian,Jakarta Pusat,LOCATION
3,Perekonomian,Senin,TIME
4,Perekonomian,Jokowi,PERSON
...,...,...,...
252082,Pertanian,Sukabumi,LOCATION
252083,Pertanian,Kabupaten Sukabumi,LOCATION
252084,Pertanian,PB Himasi,ORGANIZATION
252085,Pertanian,Dinas Peternakan Kabupaten,ORGANIZATION


In [28]:
# Hapus
mydb = mysql.connect(
        host="10.32.8.204",
        database= "berita_pdrb",
        user="dev32net",
        password="BP5jab4rb3rk1bar!"
    )

mycursor = mydb.cursor()

# QUERY POIN ADHB
sql =  f"""DELETE FROM ner;"""
    
mycursor.execute(sql)
mydb.commit()

mycursor.close()
mydb.close()

In [29]:
# Connect DB
def con_sql():
	database_username = 'dev32net'
	database_password = 'BP5jab4rb3rk1bar!'
	database_ip       = '10.32.8.204'
	database_name     = 'berita_pdrb'

	database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
												format(database_username, database_password, 
														database_ip, database_name), poolclass=NullPool)
	
	return database_connection

In [30]:
df.to_sql(con=con_sql(), name='ner', if_exists='append', index=False)

252087

In [37]:
# CLEANING

# Hapus
mydb = mysql.connect(
        host="10.32.8.204",
        database= "berita_pdrb",
        user="dev32net",
        password="BP5jab4rb3rk1bar!"
    )

mycursor = mydb.cursor()

list_person = ['Ada Festival Rame ANTVDAHANA Aktif', 'Subang Dukung Langkah Bersama Berantas', 'Subang Gelar Rekapitulasi Perhitungan Suara', 'Subang', 'Job Fair', 'Subang Job Center', 'Full Day School', ''
               'Covid', 'Cianjur', 'OPD', 'Leuwi Nangka', 'Upland Manggis', 'Nikmati Manfaat Program', 'Subang Utara', 'Ramadhan', 'Bulog', 'Idul Fitri', 'Vivo']

for person in list_person:
    print(person)
    # QUERY POIN ADHB
    sql =  f"""DELETE FROM ner WHERE entitas = '{person}' AND label = 'PERSON' ;"""
        
    mycursor.execute(sql)
    mydb.commit()

mycursor.close()
mydb.close()

Ada Festival Rame ANTVDAHANA Aktif
Subang Dukung Langkah Bersama Berantas
Subang Gelar Rekapitulasi Perhitungan Suara
Subang
Job Fair
Subang Job Center
Full Day School
Covid
Cianjur
OPD
Leuwi Nangka
Upland Manggis
Nikmati Manfaat Program
Subang Utara
Ramadhan
Bulog
Idul Fitri
Vivo


In [35]:
# CLEANING

# Hapus
mydb = mysql.connect(
        host="10.32.8.204",
        database= "berita_pdrb",
        user="dev32net",
        password="BP5jab4rb3rk1bar!"
    )

mycursor = mydb.cursor()

list_org = ['Stunting', 'ARTIKEL TERKAITPemkab', 'ARTIKEL TERKAITNgopi Bareng Media', 'ARTIKEL', 'Citarum Harum', 'Nagrak Gunakan Dana Desa Bangun', 'Indramayu', 'Jimat', 'UPLAND', 'ARTIKEL TERKAITManggis Subang Melesat',
               ';', 'PSBB', 'Depok', 'COVID', 'Bareng Media', 'Google', 'Dedi J-Budi Setiadi', 'Dedi J', 'BANDUNG', 'ASN', 'SD', 'ADVERTISEMENTKendati']

for org in list_org:
    print(org)
    # QUERY POIN ADHB
    sql =  f"""DELETE FROM ner WHERE entitas = '{org}' AND label = 'ORGANIZATION' ;"""
        
    mycursor.execute(sql)
    mydb.commit()

mycursor.close()
mydb.close()

Stunting
ARTIKEL TERKAITPemkab
ARTIKEL TERKAITNgopi Bareng Media
ARTIKEL
Citarum Harum
Nagrak Gunakan Dana Desa Bangun
Indramayu
Jimat
UPLAND
ARTIKEL TERKAITManggis Subang Melesat
;
PSBB
Depok
COVID
Bareng Media
Google
Dedi J-Budi Setiadi
Dedi J
BANDUNG
ASN
SD
ADVERTISEMENTKendati
